In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder, StandardScaler
import pickle
import pyodbc

# Database connection
conn = pyodbc.connect('DRIVER={SQL Server};'
                      'SERVER=DESKTOP-7QR5TL6;'
                      'DATABASE=DWH_SAP;'
                      'Trusted_Connection=yes')

query2 = """
SELECT
    d.Amount, VATRate, SupplierPrice,
    r.*
FROM
    [DWH_SAP].[dbo].[Dim_Supplier_Invoices] r
JOIN
    [DWH_SAP].[dbo].[Fact_Etat_Financier] d
    ON r.Pk_Supplier_Invoices = d.Fk_supplier_Invoices
"""

# Load data
df = pd.read_sql(query2, conn)

# Preprocessing
df.drop(columns=['PaymentDueDate', 'DueDate', 'InvoiceID', 'Pk_Supplier_Invoices', 'Fk_Supplier'], inplace=True)
df['log_amount'] = np.log1p(df['Amount'])
df['Approved'] = df['Approved'].astype(int)
df['PaymentStatus'] = LabelEncoder().fit_transform(df['PaymentStatus'].astype(str))

# Define features and target
X = df.drop(columns=['Approved'])
y = df['Approved']

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
num_cols = ['Amount', 'VATRate', 'SupplierPrice', 'log_amount']
X_train_scaled = X_train.copy()
X_test_scaled = X_test.copy()
X_train_scaled[num_cols] = scaler.fit_transform(X_train[num_cols])
X_test_scaled[num_cols] = scaler.transform(X_test[num_cols])

# Train the model
model = LogisticRegression(max_iter=1000)
model.fit(X_train_scaled, y_train)  # Fit on scaled data

# Save the model and scaler
with open('Amount_model1.pkl', 'wb') as file:
    pickle.dump(model, file)
with open('scalerAmount.pkl', 'wb') as file:
    pickle.dump(scaler, file)

print("Model and scaler trained and saved as Amount_model.pkl and scalerAmount.pkl")

Model and scaler trained and saved as Amount_model.pkl and scalerAmount.pkl


C:\Users\ASUS\AppData\Local\Temp\ipykernel_2192\3858086345.py:27: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query2, conn)
